<a href="https://colab.research.google.com/github/mickymultani/QuantizeLLMs/blob/main/QuantizeLLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!!git clone https://github.com/ggerganov/llama.cpp

In [ ]:
!cd llama.cpp && LLAMA_CUBLAS=1 make && pip install -r requirements.txt

In [3]:
from huggingface_hub import snapshot_download

In [4]:
model_name = "google/gemma-2b-it"

In [5]:
methods = ["q4_k_m"]

In [6]:
base_model = "./orignal_model/"

In [7]:
quantized_path = "./quantized_model/"

In [ ]:
snapshot_download(repo_id=model_name, local_dir=base_model, local_dir_use_symlinks=False)

In [9]:
orignal_model = quantized_path+"/fp16.gguf"

In [10]:
!mkdir ./quantized_model/

In [ ]:
!python llama.cpp/convert-hf-to-gguf.py ./orignal_model/ --outtype f16 --outfile ./quantized_model/FP16.gguf

In [14]:
import os

In [15]:
for m in methods:
  qtype = f"{quantized_path}/{m.upper()}.gguf"
  os.system("./llama.cpp/quantize "+quantized_path+"/FP16.gguf "+qtype+" "+m)

In [ ]:
! ./llama.cpp/main -m ./quantized_model/Q4_K_M.gguf -n 90 --repeat_penalty 1.0 --color -i -r "User: " -f llama.cpp/prompts/chat-with-bob.txt

In [18]:
from huggingface_hub import HfApi, HfFolder, create_repo, upload_file

In [19]:
model_path = "./quantized_model/Q4_K_M.gguf"
repo_name = "gemma-2b-it-GGUF-quantized"
repo_url = create_repo(repo_name, private=False)

In [20]:
api = HfApi()

In [21]:
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="Q4_K_M.gguf",
    repo_id= "mickymultani/gemma-2b-it-GGUF-quantized",
    repo_type="model",
)

Q4_K_M.gguf:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mickymultani/gemma-2b-it-GGUF-quantized/commit/92903f66bb4e303c20ff1e6262f77370172cd0ac', commit_message='Upload Q4_K_M.gguf with huggingface_hub', commit_description='', oid='92903f66bb4e303c20ff1e6262f77370172cd0ac', pr_url=None, pr_revision=None, pr_num=None)